In [1]:
# Suppress huggingface_hub warning
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="huggingface_hub")

%pip install langgraph
%pip install langchain langchain-community langchain-openai langchain-huggingface langgraph faiss-cpu sentence-transformers transformers openai tenacity pydantic numpy
%pip install --upgrade langgraph
%pip install fuzzywuzzy python-Levenshtein

# Download Chinook.sql file
!wget https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql
!head -n 100 Chinook_Sqlite.sql
# Install sqlite3
!sudo apt-get install sqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8

In [2]:
from google.colab import userdata
import getpass
import os
import os
import sqlite3

if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
    os.environ["LANGCHAIN_TRACING_V2"] = "true"

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

DB_FILE = "Chinook.db"
SQL_FILE = "Chinook_Sqlite.sql"

if not os.path.exists(DB_FILE):
    print("🔄 Chinook.db not found. Creating from Chinook_Sqlite.sql...")
    with sqlite3.connect(DB_FILE) as conn:
        with open(SQL_FILE, "r", encoding="utf-8") as f:
            conn.executescript(f.read())
    print("✅ Database created successfully.")
else:
    print("✅ Chinook.db already exists. No action needed.")

🔄 Chinook.db not found. Creating from Chinook_Sqlite.sql...
✅ Database created successfully.


In [ ]:
import os
import ast
import re
import random
import time
import json
import logging
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.prebuilt import create_react_agent
from tenacity import retry, stop_after_attempt, wait_exponential
import numpy
from fuzzywuzzy import fuzz, process
import pickle

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Step 1: Connect to DB
try:
    db = SQLDatabase.from_uri("sqlite:////content/Chinook.db")
    print("Database connected successfully")
except Exception as e:
    print(f"Database connection failed: {str(e)}")
    print("Ensure Chinook.db is uploaded to /content/Chinook.db")
    raise

# Step 2: Cache database lists
CACHE_FILE = "db_cache.json"

def query_as_list(db, query):
    try:
        res = db.run(query)
        try:
            parsed = ast.literal_eval(res)
        except (ValueError, SyntaxError):
            print(f"Error parsing query result: {res}")
            return []
        res = [el for sub in parsed for el in sub if el]
        res = [string.strip() for string in res]
        # Check for duplicates
        unique_res = list(set(res))
        if len(unique_res) < len(res):
            print(f"Warning: Duplicates found in query results for {query}")
        if not unique_res:
            print(f"Warning: No results returned for query: {query}")
        return unique_res
    except Exception as e:
        print(f"Query failed: {str(e)}")
        return []

def load_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as f:
            return json.load(f)
    return {"artists": [], "albums": [], "genres": []}

def save_cache(cache):
    with open(CACHE_FILE, 'w') as f:
        json.dump(cache, f)

# Load or query cache
cache = load_cache()
if not cache["artists"]:
    cache["artists"] = query_as_list(db, "SELECT Name FROM Artist")
    cache["albums"] = query_as_list(db, "SELECT Title FROM Album")
    cache["genres"] = query_as_list(db, "SELECT Name FROM Genre")
    save_cache(cache)

artists = cache["artists"]
albums = cache["albums"]
genres = cache["genres"]

# Verify key artists with diagnostics
key_artists = ['The Beatles', 'AC/DC', 'U2', 'Green Day', 'Queen']
for artist in key_artists:
    if artist not in artists:
        print(f"Warning: {artist} not found in Artist table.")
        # Search for similar names
        similar = process.extract(artist, artists, scorer=fuzz.token_sort_ratio, limit=3)
        if similar and similar[0][1] >= 70:
            print(f"Possible matches for {artist}: {[s[0] for s in similar if s[1] >= 70]}")

tagged_proper_nouns = (
    [f"artist::{a}" for a in artists] +
    [f"album::{a}" for a in albums] +
    [f"genre::{g}" for g in genres]
)
print("Tagged Proper Nouns:", tagged_proper_nouns)

# Step 3: LLM with retry
@retry(stop=stop_after_attempt(2), wait=wait_exponential(multiplier=1, min=2, max=5))
def create_llm():
    return ChatOpenAI(model="gpt-4o", api_key=os.environ.get("OPENAI_API_KEY"))

llm = create_llm()

# Step 4: Vector retriever
CACHE_FILE = "retriever_cache.pkl"

def load_cache():
    return pickle.load(open(CACHE_FILE, 'rb')) if os.path.exists(CACHE_FILE) else {}

def save_cache(cache):
    pickle.dump(cache, open(CACHE_FILE, 'wb'))

def preprocess_query(query: str) -> str:
    query = query.lower().strip()
    album_match = re.search(r"(?:album\s+['\"]?|released\s+['\"]?)(.+?)(?:['\"]?(\s|$)|['\"])", query, re.IGNORECASE)
    if album_match:
        return album_match.group(1).strip()
    artist_match = re.search(r"(?:by\s+|does\s+|are\s+by\s+)([\w\s&,-]+?)(?:\s+have|\s+album|\s*$)", query, re.IGNORECASE)
    if artist_match:
        return artist_match.group(1).strip()
    proper_noun_match = re.search(r'((?:[A-Z][\w\s&,-:()]+)|[\w\s&,-:()]+)', query)
    return proper_noun_match.group(1).strip() if proper_noun_match else query

def normalize_query(query: str) -> str:
  query = query.lower().strip()
  query = re.sub(r"[^\w\s&,-:()]", "", query)  # Preserve colons and parentheses
  query = re.sub(r"^(how many albums does|list the albums by|which artist released the album|which tracks belong to the|who released the album)\s+", "", query)
  query = re.sub(r"\s+(genre|have|by|album|released|the)$", "", query)
  query = re.sub(r"\s+", " ", query)
  return query.strip()

def create_custom_retriever(vector_db, proper_nouns):
    cache = load_cache()

    def custom_retriever(query: str):
        normalized_query = normalize_query(query)
        cache_key = normalized_query
        if cache_key in cache:
            print(f"Cache hit for '{query}' (normalized: '{normalized_query}')")
            return cache[cache_key]

        preprocessed_query = preprocess_query(query)
        best_match, match_score = process.extractOne(preprocessed_query, proper_nouns, scorer=fuzz.token_sort_ratio)
        if match_score >= 75:  # Lowered threshold
            print(f"Fuzzy match for '{preprocessed_query}': {best_match}, Score: {match_score}")
            result = [(best_match, 1.0 - match_score / 100)]
            cache[cache_key] = result
            save_cache(cache)
            return result

        results = vector_db.similarity_search_with_score(preprocessed_query, k=1)
        print(f"Retriever results for '{preprocessed_query}': {[doc.page_content for doc, score in results]}")
        weighted_results = [(doc.page_content, score * 0.4 if doc.page_content.startswith("artist::") else score)
                           for doc, score in results]
        weighted_results.sort(key=lambda x: x[1])

        result = weighted_results[:1]
        cache[cache_key] = result
        save_cache(cache)
        return result

    return custom_retriever

embd = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_db = FAISS.from_texts([t.lower() for t in tagged_proper_nouns], embd)
retriever = create_custom_retriever(vector_db, [t.lower() for t in tagged_proper_nouns])
retriever_tool = create_retriever_tool(
    retriever,
    "search_proper_nouns",
    "Searches for proper nouns (artist or album names) to correct misspellings."
)

# Step 5: Toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools.append(retriever_tool)

# Step 6: SQL cleaner
def clean_sql_code(sql: str) -> str:
    return re.sub(r"```(?:sql)?\s*([\s\S]*?)\s*```", r"\1", sql).strip()

# Step 7: Format result
def clean_name(name: str) -> str:
    if not name:
        return ""
    name = re.sub(r"\s*\(remastered\)|\s*\[live\]|[^\w\s&,-]", "", name, flags=re.IGNORECASE).strip()
    return name.title() if name else ""  # Capitalize for display

def format_friendly_response(question: str, resolved: str, sql: str, result: any) -> str:
    resolved_name = clean_name(resolved)
    question_lower = question.lower()

    # Extract potential artist/album/genre
    artist_match = re.search(r"(?:by\s+|does\s+|are\s+by\s+)([\w\s&,-]+?)(?:\s+have|\s+album|\s*$)", question_lower, re.IGNORECASE)
    album_match = re.search(r"album\s+['\"]?(.+?)['\"]?(\s|$)|['\"](.+?)['\"]", question_lower, re.IGNORECASE)
    genre_match = re.search(r"the\s+(.+?)\s+genre", question_lower, re.IGNORECASE)
    fallback_artist = clean_name(artist_match.group(1)) if artist_match else None
    fallback_album = clean_name(album_match.group(1) or album_match.group(3)) if album_match else None
    fallback_genre = clean_name(genre_match.group(1)) if genre_match else None

    # Fuzzy match fallbacks
    known_artists = [n.split("::")[1] for n in tagged_proper_nouns if n.startswith("artist::")]
    known_albums = [n.split("::")[1] for n in tagged_proper_nouns if n.startswith("album::")]
    if fallback_artist:
        best_match, score = process.extractOne(fallback_artist.lower(), known_artists, scorer=fuzz.token_sort_ratio)
        if score >= 75:
            fallback_artist = best_match
    if fallback_album:
        best_match, score = process.extractOne(fallback_album.lower(), known_albums, scorer=fuzz.token_sort_ratio)
        if score >= 75:
            fallback_album = best_match

    # Determine actual name from SQL result
    actual_name = None
    if sql and result and isinstance(result, list) and result:
        if sql.lower().startswith("select artist.name"):
            actual_name = result[0][0] if len(result) == 1 else ", ".join(row[0] for row in result)
        elif sql.lower().startswith("select album.title"):
            actual_name = fallback_artist or resolved_name  # Use artist for album lists
        elif sql.lower().startswith("select name from track"):
            actual_name = fallback_genre or resolved_name
        elif sql.lower().startswith("select count"):
            actual_name = fallback_artist or resolved_name

    # Prioritize actual_name, then fallbacks, then resolved_name
    display_name = clean_name(actual_name or fallback_artist or fallback_album or fallback_genre or resolved_name or "unknown")

    # Prevent raw question text
    if display_name.lower() == question_lower or display_name.lower() in question_lower:
        display_name = clean_name(fallback_artist or fallback_album or fallback_genre or "unknown")

    if result is None or (isinstance(result, list) and not result):
        if "released" in question_lower and "album" in question_lower:
            return f"🎤 No artist found for the album '{display_name}'."
        elif "album" in question_lower:
            return f"📀 No albums found for {display_name} in the database."
        elif "genre" in question_lower:
            return f"🎼 No tracks found in the {display_name} genre."
        return f"✅ No results found for {display_name}."

    items = list(set(str(r[0]) for r in result if isinstance(r, tuple) and r))
    if "list" in question_lower and "album" in question_lower:
        return f"📀 Albums by {display_name}:\n- " + "\n- ".join(items) + "\n💡 I joined Album and Artist tables to find them."
    elif "genre" in question_lower:
        header = f"🎼 Tracks in the {display_name} genre"
        if not items or items == [display_name]:
            return f"🎼 No tracks found in the {display_name} genre."
        if len(items) > 10:
            header += f" (Showing 10 of {len(items)} tracks)"
            items = items[:10]
        return f"{header}:\n- " + "\n- ".join(items) + f"\n💡 I selected tracks from the Track table where GenreId = (SELECT GenreId FROM Genre WHERE Name = '{display_name}')."
    elif "how many" in question_lower and "album" in question_lower:
        count = items[0] if items else "0"
        return f"🎧 {display_name} has {count} album(s) in the database!\n💡 I counted albums where the artist was '{display_name}'."
    elif fallback_album or "released" in question_lower:
        return f"📀 Results for {fallback_album or display_name}:\n- " + "\n- ".join(items) + "\n💡 I queried the Album and Artist tables to find the artist."
    return f"✅ Results for {display_name}:\n- " + "\n- ".join(items) + "\n💡 This result came from executing the SQL query."

# Step 8: System prompt
SQL_AGENT_PREFIX = """
You are a SQL agent for the Chinook SQLite database with tables:
- Artist (ArtistId, Name)
- Album (AlbumId, Title, ArtistId)
- Track (TrackId, Name, AlbumId, GenreId, ...)
- Genre (GenreId, Name)

Rules:
1. Use `search_proper_nouns` for artist/album names, take FIRST result.
2. For genres, check Genre table directly (e.g., SELECT Name FROM Genre WHERE Name = 'Metal'). Do NOT use `search_proper_nouns`.
3. Always call `sql_db_schema` to verify table/column names.
4. Use correct table names: 'Artist', 'Album', 'Track', 'Genre'.
5. Embed proper nouns in single quotes in SQL (no placeholders or LIKE).
6. Pass SQL to `sql_db_query_checker`. Remove markdown if present.
7. Execute with `sql_db_query`.
8. No database modifications (no INSERT/UPDATE/DELETE/DROP).
9. Return empty result for failed lookups to trigger 'No results found'.
10. Verify genre exists before querying tracks.
11. Genre names may appear in retriever suggestions as 'genre::<name>' for spelling correction, but must be verified via SQL before querying tracks.
12. If no result is found when using exact album title match, retry the query using `LIKE '%<title>%'` to allow partial album title matching.
13. Add a short explanation after the final result stating how you found the answer (e.g. which tables were used).
"""

# Step 9: Agent
system_message = SystemMessage(content=SQL_AGENT_PREFIX)
agent_executor = create_react_agent(model=llm, tools=tools, prompt=system_message)

# Step 10: Ask
def validate_retriever_result(question: str, resolved_type: str, resolved_name: str, score: float) -> tuple[str, str]:
    similarity = max(0, 1 - score)
    album_match = re.search(r"(?:album\s+['\"]?|released\s+['\"]?)(.+?)(?:['\"]?(\s|$)|['\"])", question, re.IGNORECASE)
    album_title = album_match.group(1).strip() if album_match else None
    if similarity < 0.7 and resolved_type not in ["artist", "album"]:
        return None, album_title or resolved_name
    if resolved_type == "album" and similarity < 0.5 and album_title:
        return "album", album_title
    if any(q in question.lower() for q in ["how many albums", "list the albums", "which albums are by"]):
        return "artist", resolved_name
    if "released" in question.lower() and album_title:
        return "album", album_title
    return resolved_type, resolved_name

def validate_sql_result(result: any, expected_type: str) -> bool:
    if result is None or (isinstance(result, list) and not result):
        return True  # Empty results are valid
    if not isinstance(result, list):
        return False
    if expected_type == "count":
        if (len(result) == 1 and isinstance(result[0], tuple) and len(result[0]) == 1 and
                isinstance(result[0][0], int)):
            # Reject implausible counts (e.g., >10 albums for an artist)
            return 0 <= result[0][0] <= 10
        return False
    if expected_type == "list":
        return all(isinstance(row, tuple) and len(row) >= 1 and isinstance(row[0], str)
                   for row in result)
    if expected_type == "id":
        return (len(result) == 1 and isinstance(result[0], tuple) and len(result[0]) == 1 and
                isinstance(result[0][0], int))
    return False

def detect_query_type(question: str) -> str:
    question_lower = question.lower()
    if "how many" in question_lower and "album" in question_lower:
        return "count"
    if "list" in question_lower or "which albums" in question_lower:
        return "album_list"
    if "genre" in question_lower:
        return "genre_list"
    if "which artist" in question_lower or "who released" in question_lower:
        return "artist_lookup"
    return "generic"

def ask_with_agent(question: str, max_retries=5, retry_delay=30):
    question_lower = question.lower()
    resolved_name = None
    resolved_type = None
    generated_sql = None
    final_result = None
    actual_artist = None

    # Extract potential artist/album/genre
    artist_match = re.search(r"(?:by\s+|does\s+|are\s+by\s+)([\w\s&,-]+?)(?:\s+have|\s+album|\s*$)", question_lower, re.IGNORECASE)
    album_match = re.search(r"album\s+['\"]?(.+?)['\"]?(\s|$)|['\"](.+?)['\"]", question_lower, re.IGNORECASE)
    genre_match = re.search(r"the\s+(.+?)\s+genre", question_lower, re.IGNORECASE)
    fallback_artist = clean_name(artist_match.group(1)) if artist_match else None
    fallback_album = clean_name(album_match.group(1) or album_match.group(3)) if album_match else None
    fallback_genre = clean_name(genre_match.group(1)) if genre_match else None

    # Persistent cache
    retriever_cache = load_cache()
    retriever_attempts = {}
    global_attempts = 0
    max_retriever_attempts = 2
    max_global_attempts = 4
    artist_id_cache = {}
    known_artists = [n.split("::")[1] for n in tagged_proper_nouns if n.startswith("artist::")]
    known_albums = [n.split("::")[1] for n in tagged_proper_nouns if n.startswith("album::")]

    logger.info(f"Processing question: {question}")

    # Early cache check
    normalized_query = normalize_query(question_lower)
    cache_key = normalized_query
    if cache_key in retriever_cache:
        resolved_type, resolved_name, _ = retriever_cache[cache_key]
        print(f"Cache hit for '{question}' (normalized: '{normalized_query}')")

    for attempt in range(max_retries):
        try:
            messages = [HumanMessage(content=question)]
            for step in agent_executor.stream(
                {"messages": messages},
                config={"recursion_limit": 20}
            ):
                if "tools" in step:
                    msg = step["tools"]["messages"][0]

                    if msg.name == "search_proper_nouns" and global_attempts < max_global_attempts:
                        query = question_lower
                        if 'tool_call' in msg.additional_kwargs:
                            tool_args = json.loads(msg.additional_kwargs['tool_call']['function']['arguments'])
                            query = tool_args.get("query", question).lower()
                        retriever_attempts[query] = retriever_attempts.get(query, 0) + 1
                        global_attempts += 1
                        if retriever_attempts[query] > max_retriever_attempts or global_attempts >= max_global_attempts:
                            print(f"⚠️ Reached retriever limit for '{query}'")
                            continue
                        if query not in retriever_cache:
                            best_artist_match, artist_score = process.extractOne(query, known_artists, scorer=fuzz.token_sort_ratio)
                            best_album_match, album_score = process.extractOne(query, known_albums, scorer=fuzz.token_sort_ratio)
                            if artist_score >= 70 and artist_score >= album_score:
                                results = retriever(query)
                                if results:
                                    content, score = results[0]
                                    if "::" in content:
                                        resolved_type, resolved_name = content.split("::", 1)
                                        resolved_type, resolved_name = validate_retriever_result(question, resolved_type, resolved_name, score)
                                        if resolved_type == "artist":
                                            resolved_name = best_artist_match
                                        retriever_cache[query] = (resolved_type, resolved_name, score)
                                    else:
                                        retriever_cache[query] = ("artist", best_artist_match, 1.0 - artist_score / 100)
                                else:
                                    retriever_cache[query] = ("artist", best_artist_match, 1.0 - artist_score / 100)
                            elif album_score >= 70:
                                resolved_type = "album"
                                resolved_name = best_album_match
                                retriever_cache[query] = ("album", best_album_match, 1.0 - album_score / 100)
                            else:
                                retriever_cache[query] = (None, fallback_artist or fallback_album or query, 1.0)
                            save_cache(retriever_cache)
                        resolved_type, resolved_name, _ = retriever_cache[query]
                        print(f"🔍 Resolved type: {resolved_type}")
                        print(f"🔍 Resolved name: {resolved_name}")

                    elif msg.name == "sql_db_query_checker":
                        generated_sql = clean_sql_code(msg.content)
                        query_type = detect_query_type(question)
                        if query_type == "album_list":
                            artist_name = resolved_name if resolved_type == "artist" else fallback_artist
                            if artist_name:
                                best_match, match_score = process.extractOne(artist_name.lower(), known_artists, scorer=fuzz.token_sort_ratio)
                                if match_score >= 70:
                                    artist_name = best_match
                                escaped_artist_name = artist_name.replace("'", "''")
                                generated_sql = f"SELECT Album.Title FROM Album JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Artist.Name = '{escaped_artist_name}'"
                        elif query_type == "count":
                            artist_name = resolved_name if resolved_type == "artist" else fallback_artist
                            if artist_name:
                                best_match, match_score = process.extractOne(artist_name.lower(), known_artists, scorer=fuzz.token_sort_ratio)
                                if match_score >= 70:
                                    artist_name = best_match
                                escaped_artist_name = artist_name.replace("'", "''")
                                generated_sql = f"SELECT COUNT(*) FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = '{escaped_artist_name}')"
                        elif query_type == "genre_list":
                            genre_name = resolved_name if resolved_type == "genre" else fallback_genre
                            if genre_name:
                                escaped_genre_name = genre_name.replace("'", "''")
                                generated_sql = f"SELECT Name FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = '{escaped_genre_name}')"
                        elif query_type == "artist_lookup":
                            album_name = resolved_name if resolved_type == "album" else fallback_album
                            if album_name:
                                best_match, match_score = process.extractOne(album_name.lower(), known_albums, scorer=fuzz.token_sort_ratio)
                                if match_score >= 70:
                                    album_name = best_match
                                escaped_album_name = album_name.replace("'", "''")
                                generated_sql = f"SELECT Artist.Name FROM Album JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Album.Title = '{escaped_album_name}'"
                        print(f"🧾 SQL Query: {generated_sql}")

                    elif msg.name == "sql_db_query":
                        try:
                            final_result = ast.literal_eval(msg.content) if msg.content else []
                        except (SyntaxError, ValueError) as e:
                            print(f"⚠️ Invalid SQL result format: {msg.content}, treating as empty result. Error: {str(e)}")
                            final_result = []
                        print(f"📊 SQL Result: {final_result}")
                        expected_type = "count" if "how many" in question_lower else "list" if "list" in question_lower or "which" in question_lower else "id"
                        if not validate_sql_result(final_result, expected_type):
                            print("⚠️ Invalid SQL result, retrying...")
                            if question_lower.startswith("list") or "which albums" in question_lower:
                                artist_name = resolved_name if resolved_type == "artist" else fallback_artist
                                if artist_name:
                                    best_match, match_score = process.extractOne(artist_name.lower(), known_artists, scorer=fuzz.token_sort_ratio)
                                    if match_score >= 70:
                                        artist_name = best_match
                                    escaped_artist_name = artist_name.replace("'", "''")
                                    generated_sql = f"SELECT Album.Title FROM Album JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Artist.Name = '{escaped_artist_name}'"
                                    print(f"🧾 Retry SQL Query: {generated_sql}")
                                    try:
                                        raw_result = db.run(generated_sql)
                                        final_result = ast.literal_eval(raw_result) if raw_result else []
                                        print(f"📊 Retry SQL Result: {final_result}")
                                    except (SyntaxError, ValueError) as e:
                                        print(f"⚠️ Retry query failed: {str(e)}")
                                        final_result = []
                            else:
                                final_result = None
                            continue
                        if expected_type == "list" and final_result and generated_sql:
                            if generated_sql.lower().startswith("select artist.name"):
                                actual_artist = final_result[0][0] if len(final_result) == 1 else ", ".join(row[0] for row in final_result)
                            elif generated_sql.lower().startswith("select album.title"):
                                actual_artist = resolved_name if resolved_type == "artist" else fallback_artist
                                if actual_artist:
                                    best_match, score = process.extractOne(actual_artist.lower(), known_artists, scorer=fuzz.token_sort_ratio)
                                    if score >= 70:
                                        actual_artist = best_match

                    elif msg.name == "sql_db_schema":
                        print(f"🔍 Schema Check: {msg.content[:100]}...")

                if final_result is not None:
                    break

            # Set resolved_name
            resolved_name = clean_name(actual_artist or resolved_name)
            if not resolved_name or resolved_name == question_lower:
                if fallback_artist:
                    best_match, match_score = process.extractOne(fallback_artist.lower(), known_artists, scorer=fuzz.token_sort_ratio)
                    if match_score >= 70:
                        resolved_name = best_match
                        resolved_type = "artist"
                elif fallback_album:
                    best_match, match_score = process.extractOne(fallback_album.lower(), known_albums, scorer=fuzz.token_sort_ratio)
                    if match_score >= 70:
                        resolved_name = best_match
                        resolved_type = "album"
                elif fallback_genre:
                    resolved_name = fallback_genre
                    resolved_type = "genre"

            if final_result is not None or (isinstance(final_result, list) and not final_result):
                if isinstance(final_result, list) and not final_result:
                    if fallback_album:
                        partial_album = clean_name(fallback_album)
                        print("🔁 Fallback to partial album title match...")
                        escaped_album = partial_album.replace("'", "''")
                        fallback_sql = f"SELECT DISTINCT Artist.Name FROM Album JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Album.Title LIKE '%{escaped_album}%' OR Album.Title LIKE '%{escaped_album} (Remastered)%' LIMIT 1"
                        print(f"🧾 Fallback SQL Query: {fallback_sql.strip()}")
                        try:
                            raw_result = db.run(fallback_sql)
                            fallback_result = ast.literal_eval(raw_result) if raw_result else []
                            if fallback_result and validate_sql_result(fallback_result, "list"):
                                print(f"📊 Fallback SQL Result: {fallback_result}")
                                actual_artist = ", ".join(row[0] for row in fallback_result)
                                final_result = fallback_result
                            else:
                                print("⚠️ Fallback query returned no valid results")
                        except (SyntaxError, ValueError) as e:
                            print(f"⚠️ Fallback query failed: {str(e)}")
                    if fallback_artist and resolved_type == "artist":
                        partial_artist = clean_name(fallback_artist)
                        print("🔁 Fallback to partial artist name match...")
                        if partial_artist.lower() not in artist_id_cache:
                            best_match, match_score = process.extractOne(partial_artist.lower(), known_artists, scorer=fuzz.token_sort_ratio)
                            if match_score >= 70:
                                artist_name = best_match
                                escaped_artist_name = artist_name.replace("'", "''")
                                artist_id_sql = f"SELECT ArtistId FROM Artist WHERE Name = '{escaped_artist_name}'"
                                try:
                                    artist_id_result = ast.literal_eval(db.run(artist_id_sql))
                                    if artist_id_result and validate_sql_result(artist_id_result, "id"):
                                        artist_id_cache[partial_artist.lower()] = (artist_name, artist_id_result[0][0])
                                except (SyntaxError, ValueError) as e:
                                    print(f"⚠️ Artist ID query failed: {str(e)}")
                        if partial_artist.lower() in artist_id_cache:
                            artist_name, artist_id = artist_id_cache[partial_artist.lower()]
                            album_sql = f"SELECT Title FROM Album WHERE ArtistId = {artist_id}"
                            print(f"🧾 Fallback SQL Query: {album_sql.strip()}")
                            try:
                                album_result = ast.literal_eval(db.run(album_sql))
                                if album_result and validate_sql_result(album_result, "list"):
                                    print(f"📊 Fallback SQL Result: {album_result}")
                                    actual_artist = artist_name
                                    final_result = album_result
                                else:
                                    print("⚠️ Fallback query returned no valid results")
                            except (SyntaxError, ValueError) as e:
                                print(f"⚠️ Fallback query failed: {str(e)}")

                print("✅ Final Answer:")
                display_name = clean_name(actual_artist or resolved_name or fallback_artist or fallback_album or fallback_genre or "unknown")
                print(format_friendly_response(question, display_name, generated_sql, final_result or []))
                logger.info(f"Retriever calls: {global_attempts}")
                try:
                    token_usage = step['agent']['messages'][0].response_metadata.get('token_usage', {}).get('total_tokens', 'N/A')
                    logger.info(f"Tokens used: {token_usage}")
                except:
                    logger.info("Token usage not available")
                return

        except Exception as e:
            if "429" in str(e):
                print(f"💡 Rate limit hit — retrying after {retry_delay} seconds (attempt {attempt + 1}/{max_retries}).")
                time.sleep(retry_delay)
                retry_delay *= 2
            else:
                print(f"⚠️ Error: {str(e)}")
                raise e

    print(f"⚠️ Failed after {max_retries} retries due to rate limit.")

# Step 11: Generate questions
def generate_sql_questions(db, num=5):
    questions = []
    for artist in random.sample(artists, min(num, len(artists))):
        questions.append(f"How many albums does {artist} have?")
        questions.append(f"List the albums by {artist}.")
    for genre in random.sample(genres, min(num, len(genres))):
        questions.append(f"Which tracks belong to the {genre} genre?")
    for album in random.sample(albums, min(num, len(albums))):
        questions.append(f"Which artist released the album '{album}'?")
    return random.sample(questions, min(num, len(questions)))

# Auto-test script to run all test prompts through `ask_with_agent`
test_prompts = [
    # Exact Match
    "Which artist released the album 'Supernatural'?",
    "How many albums does AC/DC have?",
    "Which tracks belong to the Metal genre?",
    "List the albums by U2.",

    # Fallback to LIKE match
    "Which artist released the album 'Unplugged'?",
    "Which artist released the album 'Diary of a Madman'?",
    "Who released the album 'Great Recordings of the Century'?",

    # Misspellings (retriever test)
    "How many albums does Greeen Day have?",
    "List the albums by The Beatlz",

    # No match (negative case)
    "How many albums does Unknown Band XYZ have?",
    "Which artist released the album 'Completely Made Up Album Title'?",

    # Ambiguity and real-world phrasing
    "Which albums are by Queen?",
    "Which artist released 'American Idiot'?"
]

# Step 12: Run
if __name__ == "__main__":
    questions = generate_sql_questions(db, num=3)
    for q in questions:
        logger.info(f"Running generated question: {q}")
        ask_with_agent(q)
        print("----")
        time.sleep(30)  # Increased delay to avoid rate limits
    for q in test_prompts:
        logger.info(f"Running test prompt: {q}")
        ask_with_agent(q)
        print("----")
        time.sleep(30)  # Increased delay to avoid rate limits

Database connected successfully
Tagged Proper Nouns: ['artist::Deep Purple', 'artist::Los Hermanos', 'artist::Antônio Carlos Jobim', 'artist::Edson, DJ Marky & DJ Patife Featuring Fernanda Porto', 'artist::Joe Satriani', 'artist::BackBeat', 'artist::Frank Zappa & Captain Beefheart', 'artist::Toquinho & Vinícius', 'artist::Nação Zumbi', 'artist::Chicago Symphony Chorus, Chicago Symphony Orchestra & Sir Georg Solti', 'artist::Vinícius De Moraes', 'artist::Maurizio Pollini', 'artist::Nirvana', 'artist::Def Leppard', 'artist::Milton Nascimento', 'artist::Karsh Kale', 'artist::Rodox', 'artist::Aaron Copland & London Symphony Orchestra', 'artist::Gene Krupa', 'artist::The Posies', 'artist::Motörhead & Girlschool', 'artist::Eugene Ormandy', 'artist::James Brown', 'artist::Elis Regina', 'artist::O Rappa', 'artist::Marcos Valle', 'artist::Big & Rich', 'artist::Caetano Veloso', 'artist::Luciana Souza/Romero Lubambo', 'artist::London Symphony Orchestra & Sir Charles Mackerras', 'artist::Metallica